# Fast Fourier Transform (FFT)

In [ ]:
import timeit

import matplotlib.pyplot as plt
import numpy as np
from numpy.typing import NDArray
from scipy import signal
from scipy.fft import fft, fftshift, irfft, next_fast_len, rfft, rfftfreq

In the notebook on interpolation, we saw that we can model data using (piecewise) polynomial basis functions. When modeling *cyclical* data, it is more informative to use trigonometric functions (sines and cosines) as basis function.

Representing a function as a linear combination of sines and cosines decomposes the function into its components of various frequencies, similarly to how a prism resolves a light beam into its constituent colors. The resulting coefficients reveal which frequencies are present in the function/data and in what amounts.

Moreover, we'll see that the representation of the function/data in *frequency domain* (as opposed to *time domain*) makes it possible to perform manipulations necessary in signal processing, or in solving differential equations in a very efficient way.

The Fourier transform in computational sciences is a vast topic by itself, with extensive specialized literature. For example, the book [The Fourier transform and its applications](https://www.worldcat.org/title/fourier-transform-and-its-applications/oclc/1025024564) by Ronald N. Bracewell is a classic reference.

## Notation

We'll use complex exponential notation because it will allow us to write down the equations in a convenient way.

Recall **Euler's identity**

$$ e^{i\theta}=\cos\theta +i \sin\theta$$

with $i=\sqrt{-1}$.

This allows us to write a cosine or sine wave with frequency $f$ as

$$
\begin{aligned}
    \cos(2\pi ft) &= \frac{e^{2\pi i ft}+e^{-2\pi i ft}}{2} \\
    \sin(2\pi ft) &= \frac{e^{2\pi i ft}-e^{-2\pi i ft}}{2i}
\end{aligned}
$$

Now let's introduce the notation

$$\omega_n=\cos(2\pi/n)-i\sin(2\pi/n)=e^{-2\pi i/n}$$

for the (or rather *one particular*) $n^\text{th}$ root of unity, meaning that $\omega^n_n=1$. 

Also note the effect of complex conjugation: $\omega^*_n = \omega^{-1}_n = \omega^{n-1}_n$.

## Discrete Fourier Transform

### Definition

Given a sequence $\mathbf{x}=[x_0,\ldots,x_{n-1}]^\intercal$, its **discrete Fourier transform** (or DFT), is the sequence $\mathbf{y}=[y_0,\ldots,y_{n-1}]^\intercal$ given by 

$$y_m=\sum_{k=0}^{n-1}\omega_n^{mk}x_k,\quad \forall\,m\in\{0,1,\ldots,n-1\}$$

This can be written in matrix notation as $\mathbf{y}=\mathbf{F}_n\mathbf{x}$ where the entries of the symmetric Fourier matrix $\mathbf{F}_n$ are given by

$$\{\mathbf{F}_n\}_{mk}=\omega_n^{mk}$$

> **Example**
>
> For $n=4$ the Fourier matrix is given by =
>
> $$
\begin{aligned}\mathbf{F}_4=\begin{bmatrix}
1&1&1&1\\
1&\omega^1&\omega^2&\omega^3\\
1&\omega^2&\omega^4&\omega^6\\
1&\omega^3&\omega^6&\omega^9\end{bmatrix}=
\begin{bmatrix}
1&1&1&1\\
1&-i&-1&i\\
1&-1&1&-1\\
1&i&-1&-i\end{bmatrix}\end{aligned}
$$
>
> where the subscript $4$ from $\omega_4$ was dropped for visual clarity.

> **Visualization**
>
> The following code visualizes the Fourier matrix for $n = 32$.
> Since all matrix elements are complex values whose absolute value is $1$, the color code just represents the phase (expressed in radians).

In [ ]:
def plot_phases(n=32):
    """Visualize the phase of elements of the Fourier matrix."""
    # Make a matrix with all the phases.
    irow, icol = np.mgrid[:n, :n]
    tau = 2 * np.pi
    phase = tau * (irow * icol) / n
    phase -= np.round(phase / tau) * tau

    # Plot the matrix.
    plt.close("phase")
    fig, ax = plt.subplots(num="phase", figsize=(5, 4))
    ms = ax.matshow(phase, cmap="hsv")
    cbar = fig.colorbar(ms)
    cbar.set_label("phase")
    cbar.set_ticks([-np.pi, -np.pi / 2, 0, np.pi / 2, np.pi])
    cbar.set_ticklabels([r"$-\pi$", r"$-\pi / 2$", "$0$", r"$\pi / 2$", r"$\pi$"])
    ax.spines["top"].set_visible(True)
    ax.spines["bottom"].set_visible(False)
    ax.tick_params(axis="x", which="both", bottom=False)


plot_phases()

The inverse of $\mathbf{F}_n$ is given by $\mathbf{F}_n^{-1}=(1/n)\mathbf{F}_n^H$. 

> **Short Proof**
>
> $$
\begin{aligned}
\{\mathbf{F}_n\mathbf{F}_n^{-1}\}_{mm}
&= \frac{1}{n} \sum_{k=0}^{n-1} \omega_n^{mk} (\omega_n^*)^{km} = 1 \\
\{\mathbf{F}_n\mathbf{F}_n^{-1}\}_{m\ell}
&= \frac{1}{n} \sum_{k=0}^{n-1} \omega_n^{mk} (\omega_n^*)^{k \ell} = \frac{1}{n} \sum_{k=0}^{n-1} \omega_n^{(m-\ell)k} = \frac{1}{n} \frac{1 - \omega_n^{n(m-\ell)}}{1-\omega_n^{m-\ell}} = 0
\end{aligned}
$$
>
> In the last step, we made use of $\omega_n^{n} = 1$.

The **inverse** DFT thus reads

$$
\begin{aligned}
    x_k
    &= \frac{1}{n}\sum_{m=0}^{n-1}\omega_n^{-km}y_m \\
    &= \frac{1}{n}\sum_{m=0}^{n-1}y_m\bigl[\cos(2\pi mk / n) + i \sin(2\pi mk / n) \bigr],
    & \forall\,k\in\{0,1,\ldots,n-1\}
\end{aligned}
$$

This inverse transform represents the components of $\mathbf{x}$ as a linear combination of sines and cosines, with coefficients given by the components of $\mathbf{y}$.
Thus, if the components of $\mathbf{x}$ are sample values of a function, then the DFT solves the trigonometric interpolation problem with nothing more than a matrix-vector multiplication.

The cost scales as $\mathcal{O}(n^2)$ instead of $\mathcal{O}(n^3)$ when solving a linear system, as we saw in the interpolation notebook.
Better still, using the **Fast Fourier transform** algorithm, this can be reduced even further to $\mathcal{O}(n\log(n))$.

### Frequencies

In the definitions of the (inverse) DFT, the frequency is represented by a dimensionless integer index $m$, which seems to have the wrong unit.
It is implicitly assumed that the samples $x_k$ are taken at regular (time) steps

$$t_k = t_0 + k / f_s.$$

where $f_s$ is called the sampling rate.
Put differently,

$$k = (t_k - t_0) f_s.$$

With that choice, the sine and cosine basis functions, can be written as a function of time:

$$
\begin{aligned}
    \omega_n^{-mk}
    &= \cos(2\pi mk/n) + i\sin(2\pi mk/n) \\
    &= \cos\left(2\pi\frac{f_s  m}{n} (t_k - t_0)\right) + i\sin\left(2\pi\frac{f_s  m}{n} (t_k - t_0)\right)
\end{aligned}
$$

As soon as a sampling rate $f_s$ is assumed, the index $m$ corresponds to a genuine frequency $\frac{f_s m}{n}$.

Note that one may also sample in a spatial domain instead of the time domain, in which case the index $m$ corresponds to a wavenumber instead of a frequency.

### Negative frequencies

One classical source of confusion is the terminology of *negative frequency* when interpreting a DFT.
This seems wrong because the inverse DFT only makes use of a positive index $m$, such that all cosine and sine functions have positive frequencies:

$$
\begin{aligned}
    x_k
    &= \frac{1}{n}\sum_{m=0}^{n-1}\omega_n^{-km}y_m \\
    &= \frac{1}{n}\sum_{m=0}^{n-1}y_m\bigl[\cos(2\pi mk / n) + i \sin(2\pi mk / n) \bigr],
    & \forall\,k\in\{0,1,\ldots,n-1\}
\end{aligned}
$$

However, one can always make use of $\omega_n^{-mk} = \omega_n^{(\ell n-m)k}$, where $\ell$ is an arbitrary integer, because $\omega_n^n=1$. With this, the inverse DFT can be rewritten in many different froms:

$$x_k=\frac{1}{n}\sum_{m=0}^{n-1}y_{m}\omega_n^{(n-m)k}, \quad \forall\,k\in\{0,1,\ldots,n-1\}$$

or 

$$x_k=\frac{1}{n}\sum_{m=0}^{n-1}y_{m}\omega_n^{(-n-m)k}, \quad \forall\,k\in\{0,1,\ldots,n-1\}$$

etc.
This shows that the coefficient $m$ is not associated with a single frequency, but rather with an infinite number of them.

For practical interpretations, one limits the infinite possibilities to those frequencies (either with positive or negative sign) with the smallest absolute value. The sequence of indexes $m$ is

$$[0, \ldots, n-1]$$

and corresponding frequencies for interpretative purposes, assuming a sampling rate $f_s$, are

$$
\left[
    0,
    \frac{f_s}{n},
    \frac{2 f_s}{n},
    \ldots,
    \frac{\lfloor n/2 \rfloor f_s}{n},
    \frac{(\lfloor n/2 \rfloor - n) f_s}{n},
    \ldots,
    \frac{-2 f_s}{n},
    \frac{-f_s}{n},
\right]
$$

where $\lfloor \cdot \rfloor$ represents the floor function.
It rounds towards a lower (more negative) integer.
In SciPy, the function [`fftfreq`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.fftfreq.html#scipy.fft.fftfreq) constructs this type of frequency axis.

If you have trouble with the meaning of a negative frequency, think of positive frequencies as *clockwise* and negative frequencies as *counterclockwise*.

> **Illustration with sinusoidal input**
> 
> The following code cell illustrates basic DFT concepts.
> The input for the DFT is a sinusoidal function, of which you can control the frequency, phase, sampling rate and number of samples.

In [ ]:
def compute_fft_demo_data(
    freq0: float, sampling_rate: float, phase: float, nsample: int
) -> tuple[NDArray, NDArray, NDArray, NDArray]:
    """Generate a sinusoidal signal and its FFT.

    Parameters
    ----------
    freq0
        Frequency of the sinusoidal signal.
    sampling_rate
        Sampling rate.
    phase
        Phase shift of the signal.
    nsample
        Number of samples.

    Returns
    -------
    time_axis
        Time axis.
    signal_time
        Sinusoidal signal.
    freq_axis
        Frequency axis.
    signal_freq
        Discrete Fourier Transform of the signal.
    """
    # All values are in SI base units.
    # This example is put in a function to avoid that
    # the local variables interfere with the examples
    # below.

    # 'Sample' a sine signal
    # and compute its DFT.
    time_axis = np.arange(nsample) / sampling_rate
    signal_time = np.sin(2 * np.pi * freq0 * time_axis + phase)
    signal_freq = fft(signal_time)

    # For the sake of plotting the FFT, construct a frequency
    # axis in which we make some frequencies explicitly negative.
    freq_axis = np.arange(nsample) * sampling_rate / nsample
    freq_axis[nsample // 2 + 1 :] -= sampling_rate
    # One might also use
    # freq_axis = fftfreq(nsample, 1/sampling_rate)

    return time_axis, signal_time, freq_axis, signal_freq


def fft_demo(sampling_rate, freq0, phase, nsample, num):
    time_axis, signal_time, freq_axis, signal_freq = compute_fft_demo_data(
        freq0, sampling_rate, phase, nsample
    )
    plt.close(num)
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(8, 4), num=num, clear=True)

    # Create sine signal for reference (in meters as a function of time in seconds)
    # Sine signal is plotted till last sample point
    t = np.linspace(0, time_axis[-1], 10000)

    # Plot the sine signal and sample functions
    ax0.plot(
        t, np.sin(2 * np.pi * freq0 * t + phase), "-", color="blue", label="Signal"
    )
    ax0.scatter(
        time_axis, signal_time, color="red", label="Sample Points", s=20, zorder=5
    )
    ax0.plot(time_axis, signal_time, ":", color="red", label="Sample Output")
    ax0.set_xlim(0, 5)
    ax0.set_xlabel("Time [s]")
    ax0.set_ylabel("Time-dependent position [m]")
    ax0.legend(loc="upper right")

    # Create a list of all frequency values (used in set_ylim)
    all_freq = signal_freq.real + signal_freq.imag

    # Plot the DFT
    ax1.scatter(freq_axis, signal_freq.real, s=15, label="Real part", zorder=5)
    ax1.scatter(freq_axis, signal_freq.imag, s=15, label="Imag part", zorder=5)
    ax1.axvline(-freq0, color="k", label=f"Signal frequency ({freq0:.2f} Hz)")
    ax1.axvline(freq0, color="k")
    ax1.set_xlabel("Frequency [Hz]")
    ax1.set_ylabel("Discrete Fourier Transform [m]")
    ax1.set_ylim([min(all_freq) - 5, max(all_freq) + 5])
    ax1.legend()
    ax1.grid()

    fig.suptitle("Discrete Fourier Transform (DFT) Demo")

> Let's first consider a sine input. It can be decomposed as:
>
> $$\sin(2\pi f t) = \frac{i}{2}\exp(-i 2\pi f t) - \frac{i}{2}\exp(+i 2\pi f t)$$
>
> This corresponds to two imaginary non-zero values in the descrete Fourier transform.

In [ ]:
fft_demo(sampling_rate=4, freq0=1, phase=0, nsample=256, num="case1_sine")

> We can also look at the result for a similar cosine, which can be decomposed as:
>
> $$\cos(2\pi f t) = \frac{1}{2}\exp(-i 2\pi f t) + \frac{1}{2}\exp(+i 2\pi f t)$$
>
> This corresponds to two identical real non-zero values in the descrete Fourier transform.

In [ ]:
fft_demo(sampling_rate=4, freq0=1, phase=np.pi / 2, nsample=256, num="case2_cosine")

> Finally, we can analyze a sine function with a fractional number of periods in the sequence.
> The DFT has many non-zero values around the expected frequency.
> This phenomenon is called [spectral leakage](https://en.wikipedia.org/wiki/Spectral_leakage).

In [ ]:
fft_demo(sampling_rate=4, freq0=0.5246, phase=0, nsample=256, num="case3_leakage")

> Note that the DFT is computed with the FFT algorithm from SciPy.
> This algorithm will be explained in the next section, but how it works is not important for this example.
> For now, only the result matters.

### Nyquist frequency & Nyquist-Shannon sampling theorem

In the sequence of frequencies shown above, the highest one is $\lfloor n/2 \rfloor f_s / n$.
In the limit of many samples, or for any even number of samples, the highest frequency always becomes $f_s / 2$, which is known as the [Nyquist frequency](https://en.wikipedia.org/wiki/Nyquist_frequency).

If the underlying time-dependent function, of which $x_k$ are samples, contains relevant fluctuations at frequencies above $f_s / 2$, it will be impossible to discern them from lower-frequency fluctuations.
The reason is that, on the sampling grid, the basis function $\omega_n^{mk}$ is indistinguishable from $\omega_n^{(\ell n + m)k}$, where $\ell$ is an arbitrary integer.

In practice, this means that the sampling frequency should at least twice the highest relevant frequency of the underlying function. This is known as the [Nyquist-Shannon sampling theorem](https://en.wikipedia.org/wiki/Nyquist%E2%80%93Shannon_sampling_theorem).

> **Example**
>
> As shown by the `scipy` example below, the DFT of the sequence $[+\!1\, -\!1\, +\!1 -\!1\, +\!1\, -\!1\, +\!1\, -\!1]^\intercal$, which has the highest possible rate of oscillation is given by 
>
> $$
\begin{aligned}
\mathbf{F}_8\mathbf{x}=\mathbf{F}_8\begin{bmatrix}
1 \\ -1 \\ 1 \\ -1 \\ 1 \\ -1 \\ 1 \\ -1
\end{bmatrix}=\begin{bmatrix}
0 \\ 0 \\ 0 \\ 0 \\ 8 \\ 0 \\ 0 \\ 0
\end{bmatrix}
\end{aligned}
$$
>
> In the transformed sequence, we see only a nonzero component at the Nyquist frequency ($y_4$), and $y_0$ indeed equals the sum of the elements of $\mathbf{x}$.

In [ ]:
fft([1, -1, 1, -1, 1, -1, 1, -1])

> **Illustration with the `fft_demo` function (Nyquist & Shannon)**
>
> In the following cell, the `fft_demo` is executed with a frequency $f_0=f_s + 0.5$.
> In line with the theory, the results look identical to the case of $f_0 = 0.5$.
> Try to see what happens when $f_0 = f_s / 2 + 0.5$.
> Can you explain the result?

In [ ]:
fft_demo(sampling_rate=4, freq0=4.5, phase=0, nsample=256, num="case4_nyquist")

### The DC component (zero frequency)

The lowest of all frequencies is always zero (in $\mathrm{Hz}$ or $\mathrm{m}^{-1}$).
The corresponding coefficient of the DFT is simply the sum of all values $x_k$ and is called the DC (direct current) component.

Note that a constant shift of all $x_k$ will only affect the DC component.

> **Simple illustration**
>
> The following two inputs to the `fft` function only differ by a constant shift, which is reflected in (only) a difference in DC component.

In [ ]:
fft([2, 0, 2, 0, 2, 0, 2, 0])

In [ ]:
fft([1, -1, 1, -1, 1, -1, 1, -1])

### DFT of a real sequence

The DFT of a sequence (even a real sequence) is in general complex.
This is not something to worry about, as the inverse DFT will take us back to the real domain.

The DFT of a real sequence of length $n$ has $2n$ real and imaginary parts, but still contains only $n$ independent pieces of information.
This can be understood as follows: for each basis function $\omega_n^{-km}$, there is another basis function $\omega_n^{k(n+m)} = \omega_n^{km} = (\omega^*)_n^{-km}$, which is just its complex conjugate.
To represent a real sequence $\mathbf{x}$, the DFT must adhere to $y_m = y^*_{n-m}$.

> **Short Proof**
>
> $$
\begin{aligned}
y_{n-m}^*
  &= \biggl(\sum_{k=0}^{n-1} \omega_n^{k(n-m)} x_k\biggr)^*
   = \biggl(\sum_{k=0}^{n-1} \omega_n^{kn} \omega_n^{-km} x_k\biggr)^*
   = \biggl(\sum_{k=0}^{n-1} \omega_n^{-km} x_k\biggr)^* \\
  &= \sum_{k=0}^{n-1} (\omega_n^*)^{-km} x_k^*
   = \sum_{k=0}^{n-1} \omega_n^{km} x_k^*
   = \sum_{k=0}^{n-1} \omega_n^{km} x_k
   = y_m
\end{aligned}
$$
>
> In the third step, we made use of $\omega_n^{kn} = 1$.
> 
This has a few implications for real sequences:

- $y_0$ is real as it is just equal to the sum of the real components $x_k$.
- When $n$ is even, $y_{n/2}$ is also real.
- When $n$ is odd, $y_{(n+1)}/2$ may still be complex.

Because the second half of a DFT of a real sequence is redundant, a specialized function `rfft` exists to compute only the first half of the DFT.

> **Simple illustration**
>
> The following uses as input a real sequence, and as expected, the last part of the DFT contains redundant information.

In [ ]:
fft([1.2, 2.5, 3.1, 2.3, 1.4, 1.0])

> The `rrft` function only accepts real inputs and is faster because it skips the redundant computations for real inputs.

In [ ]:
rfft([1.2, 2.5, 3.1, 2.3, 1.4, 1.0])

> The following illustrates that the last element of the `rfft` function corresponds to the Nyquist frequency.

In [ ]:
rfft([1, -1, 1, -1, 1, -1, 1, -1])

## FFT Algorithm

We can exploit certain symmetries and redundancies in the definition of the DFT to calculate it efficiently, with a cost scaling like $\mathcal{O}(n\log(n))$ instead of $\mathcal{O}(n^2)$.
This will first be illustrated with an example for $n=4$:

> **Example**
>
> From the definition of the DFT, we have
>
> $$y_m=\sum^3_{k=0}x_k\omega_4^{mk},\quad m=0,\ldots,3$$
>
> Again, for the sake of visual clarity, we will use $\omega=\omega_4$ below.
>
> Writing this out in full, we have
> 
> $$
\begin{aligned}
    y_0 &= x_0\omega^0 + x_1\omega^0 + x_2\omega^0 + x_3\omega^0 \\
    y_1 &= x_0\omega^0 + x_1\omega^1 + x_2\omega^2 + x_3\omega^3 \\
    y_2 &= x_0\omega^0 + x_1\omega^2 + x_2\omega^4 + x_3\omega^6 \\
    y_3 &= x_0\omega^0 + x_1\omega^3 + x_2\omega^6 + x_3\omega^9
\end{aligned}
$$
>
> We know that
> 
> $$
\begin{aligned}
    \omega^0 &= \omega^4=1 \\
    \omega^2 &= \omega^6=-1 \\
    \omega^9 &= \omega^1
\end{aligned}
$$
>
> so we can regroup the equations as
>
> $$
\begin{aligned}
    y_0 &= (x_0+x_2)+\omega^0(x_1+x_3) \\
    y_1 &= (x_0-x_2)+\omega^1(x_1-x_3) \\
    y_2 &= (x_0+x_2)+\omega^2(x_1+x_3) \\
    y_3 &= (x_0-x_2)+\omega^3(x_1-x_3)
\end{aligned}
$$
>
> This shows that computing the DFT of the original 4 point sequence has been reduced to computing the DFT of its 2 point even and odd subsequences:
>
> $$
\begin{aligned}
    \tilde{y}_0^\text{even} &= x_0 + \omega_2^0 x_2 \\
    \tilde{y}_1^\text{even} &= x_0 + \omega_2^1 x_2 \\
    \tilde{y}_0^\text{odd} &= x_1 + \omega_2^0 x_3 \\
    \tilde{y}_1^\text{odd} &= x_1 + \omega_2^1 x_3
\end{aligned}
$$
>
> and
>
> $$
\begin{aligned}
    y_0 &= \tilde{y}_0^\text{even} + \omega^0\tilde{y}_0^\text{odd} \\
    y_1 &= \tilde{y}_1^\text{even} + \omega^1\tilde{y}_1^\text{odd} \\
    y_2 &= \tilde{y}_0^\text{even} + \omega^2\tilde{y}_0^\text{odd} \\
    y_3 &= \tilde{y}_1^\text{even} + \omega^3\tilde{y}_1^\text{odd}
\end{aligned}
$$

This property holds in general: the DFT of an $n$-point sequence can be computed by breaking it into two DFTs of half the length, provided $n$ is even.

> **Example (continued)**
> 
> In matrix notation, this becomes more clear.
> Consider the first few Fourier matrices:
> 
> $$\mathbf{F}_1=1$$
>
> $$
\begin{aligned}\mathbf{F}_2=\begin{bmatrix}1&1\\1&-1\end{bmatrix}\end{aligned}
$$
>
>  $$
\begin{aligned}\mathbf{F}_4=\begin{bmatrix}
1&1&1&1\\
1&-i&-1&i\\
1&-1&1&-1\\
1&i&-1&-i\end{bmatrix}\end{aligned}
$$
>
> Additionally let $\mathbf{P}_4$ be the permutation matrix, which separates the odd and even subsequences:
>
> $$
\begin{aligned}\mathbf{P}_4=\begin{bmatrix}1&0&0&0\\
0&0&1&0\\
0&1&0&0\\
0&0&0&1\end{bmatrix}\end{aligned}
$$
>
> and $\mathbf{D}_2$ the diagonal matrix
>
> $$
\begin{aligned}\mathbf{D}_2=\mathrm{diag}(1,\omega_4^1)=\begin{bmatrix}1&0\\0&-i\end{bmatrix}\end{aligned}
$$
>
>
> With these matrices, we can rearrange $\mathbf{F}_4$ such that each block is a diagonally scaled version of $\mathbf{F}_2$.
>
>
>$$
\begin{aligned}\mathbf{F}_4\mathbf{P}_4=\left[\begin{array}{c c|c c} 
	1 & 1 & 1 & 1\\
    1 &-1 & -i & i\\
	\hline 
	1&1&-1 &-1\\
    1&-1&i&-i
\end{array}\right]=\begin{bmatrix}
    \mathbf{F}_2&\mathbf{D}_2\mathbf{F}_2\\\mathbf{F}_2&-\mathbf{D}_2\mathbf{F}_2
\end{bmatrix}\end{aligned}
$$

In general, $\mathbf{P}_n$ is the permutation that groups the even-numbered columns of $\mathbf{F}_n$ before the odd numbered columns and 

$$\mathbf{D}_{n/2}=\mathrm{diag}\left(1,\omega_n,\ldots,\omega_n^{(n/2)-1}\right)$$

Thus, to apply $\mathbf{F}_n$ to a sequence of length $n$, we merely need to apply $\mathbf{F}_{n/2}$ to its even and odd subsequences and scale the results by $\pm\mathbf{D}_{n/2}$. 
This recursive *divide-and-conquer* approach is called the **fast Fourier transform** and can be used to recursively calculate the DFT of sequences of any length (that is a power of 2).

Despite its name, the fast Fourier transform is an algorithm and not a transform.
It's a specific way of efficiently calculating the discrete Fourier transform.
There are $\log_2(n)$ levels of recursion, each of which involves $\mathcal{O}(n)$ operations, so the total cost of the FFT scales as $\mathcal{O}(n\log(n))$.
The following table shows that, for large sequences, this makes a huge practical difference compared to the $\mathcal{O}(n^2)$ required for a straightforward implementation of the DFT:

$$
\begin{array}{|c|c|c|}
\hline
 n  	& n\log_2(n) 	& n^2   	\\ \hline
 8    	& 24           	& 64      	\\
 16   	& 64           	& 256     	\\
 32   	& 160          	& 1024    	\\
 64   	& 384          	& 4096    	\\
 128  &	 896          	& 16384   	\\
 256 & 	 2048         	& 65536   	\\
 512  &	 4608         	& 262144  	\\
 1024 &	 10240        	& 1048576 	\\ \hline
\end{array}
$$

> **History**
> 
>The reason why scientists started working on finding a method to calculate Fourier Transforms more efficiently with less 
operations goes back to the aftermath of World War II. After the USA used their atomic bomb to shock the whole world, a lot of other countries started testing nuclear weapons for further use. After a convention in Geneva the big nations agreed to ban further testing in the atmosphere and under water, but not underground. Why? It's hard to detect if an atomic bomb is released underground, but by decomposing seismic signals it can be analyzed! Sadly the discovery of the FFT algorithm was too late to prevent an all out nuclear race. This means if it was invented earlier the Cold War maybe never happened!!
>
> Source: *Veritasium. (2022a, november 3). The Remarkable Story Behind The Most Important Algorithm Of All Time [Youtube Video](https://www.youtube.com/watch?v=nmgFG7PUHfo).
>
> Since its discovery by Cooley and Tukey in 1965, the practical value of the Fast Fourier Transform algorithm has been immense for many fields of research and applications. 
> However, it was already discovered by Carl Friedrich Gauss 160 years earlier, but somehow was forgotten again!


### Limitations and extensions

Although very efficient for some use cases, the FFT algorithm has a few limitations:
- The input sequence needs to be **equally spaced** (although this sometimes can be mitigated using interpolation).
- The input sequence is assumed to be **periodic** (resulting from the definition of the DFT, which tries to transform the data in a linear combination of sines and cosines).
  
  > In signal processing, the discrete cosine transform (DCT) is often used instead of DFT.
  > It does not assume the input is periodic, making it more broadly applicable.
  > Furthermore, the DCT of a real signal is also real, making the output more intuitive.
  > The DCT is also implemented in SciPy, with algorithms very similar to the FFT.

- The sequence is assumed to be a **power-of-2 in length** (specific to our algorithm).

  > The FFT can be generalized to handle sequences of arbitrary length, not only powers of 2.
  > The general FFT algorithm does not only split a sequence into two.
  > Instead, it partitions a sequence in $M\ge 2$ subsets, where $M$ is the smallest prime factor of the length of a sequence, at the current recursion level.
  > So in general, $M$ may vary across different levels of recursion.
  > At each level, an $M$-fold DFT must be computed with conventional matrix-vector multiplication.
  >
  > This shows that the FFT, as discussed here, will be slow for long sequences whose length is a prime number.
  > For such cases, more advanced implementations exist to efficiently handle sequences whose lengths are prime numbers.
  > These advanced algorithms still result in an $\mathcal{O}\left(n\log(n)\right)$ scaling, but with a much larger prefactor, compared to the algorithm discussed here.

### Windowing (elective, can be skipped)

Windowing is employed to address a specific challenge associated with FFT: spectral leakage.
This phenomenon occurs when analyzing finite segments of a signal, causing inaccuracies in frequency domain representation.
For an overview of various windowing functions, you can explore Window function on [Wikipedia](https://en.wikipedia.org/wiki/Window_function).

Mathematically, applying a window involves multiplying the original signal by a window function, $w_k$. One common window function is the cosine window, expressed as:

$$w_k = \frac{1}{2}-\frac{1}{2}\cos \left(\frac{2\pi k}{n-1} \right)$$

Before applying the discrete Fourier transform, the signal is first multiplied with the window:

$$\tilde{x}_k = w_k x_k$$

Here, $\tilde{x}_k$ represents the windowed signal, $k$ is the sample index, and $n$ is the total number of samples.
The window function gently tapers the signal at its edges, reducing abrupt changes and mitigating spectral leakage.

Windowing offers numerous benefits, including reduced spectral leakage, improved frequency resolution, and control over side lobe suppression.
The choice of a specific window function depends on the requirements of the signal processing task at hand and the characteristics desired in the frequency domain analysis.

The following example shows a typical plot of a window function.
To characterize the window function, it can be Fourier transformed as such, which effectively shows the result of windowing a constant function.
The Fourier transform shows a central peak at frequency zero (corresponding to the constant signal), which has a certain width and side lobes due to the fact that the function is truncated.
When the `cosine` window by another function from [`scipy.signal.windows`](https://docs.scipy.org/doc/scipy/reference/signal.windows.html#module-scipy.signal.windows), the shape of the main peak and the side lobes in the spectrum will change.

In [ ]:
def demo_windowing():
    window = signal.windows.cosine(51)
    # window = signal.windows.blackman(51)
    # window = signal.windows.bartlett(51)

    plt.close("windowing")
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), num="windowing")
    ax1.plot(window)
    ax1.set_title("Cosine window")
    ax1.set_ylabel("Amplitude")
    ax1.set_xlabel("Sample")

    A = fft(window, 2047) / (len(window) / 2.0)
    freq = np.linspace(-0.5, 0.5, len(A))
    response = 20 * np.log10(np.abs(fftshift(A / abs(A).max())))
    ax2.plot(freq, response)
    ax2.axis([-0.5, 0.5, -120, 5])
    ax2.set_title("Frequency response of the cosine window")
    ax2.set_ylabel("Normalized magnitude [dB]")
    ax2.set_xlabel("Normalized frequency [cycles per sample]")


demo_windowing()

### Performance demo

In the following cells, we're going to perform the FFT of a sequence with lengths ranging from 1 to 4100, and track the required CPU time.
Next, we're going to repeat this, but now by zero-padding each sequence until the length given by [`next_fast_len`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fft.next_fast_len.html).

In the resulting figures, you'll see that powers of 2 are indeed the most efficient, and that prime numbers are the least efficient lengths, even for the optimized `scipy` implementation. However, zero padding the input until `next_fast_len` mitigates part of these problems (although the zero padding in itself might introduce spurious noise in your Fourier transform!)

**Sequence of prime numbers**

We'll first make a little detour and show how to construct a sequence of prime numbers using vectorization techniques in NumPy.
The algorithm is based on [Eratosthenes' sieve](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes).

In [ ]:
def construct_primes(nabove: int) -> np.ndarray:
    """Return an array with prime numbers below ``nabove``."""
    # This is the Python equivalent of the sieve of Eratosthenes.
    sieve = np.ones(nabove, bool)
    sieve[:2] = False
    divisor = 2
    maxdivisor = np.sqrt(nabove)
    while divisor < maxdivisor:
        if sieve[divisor]:
            sieve[2 * divisor :: divisor] = False
        divisor += 1
    return sieve.nonzero()[0]


assert (construct_primes(10) == [2, 3, 5, 7]).all()
print(construct_primes(50))

**Timing function**

The following cell implements a function that times the `fft` function with random input data for a series of lengths. 

In [ ]:
def time_fft(maxlen: int, pad: bool = False) -> np.ndarray:
    """Time the fft function for a series of input sizes.

    Parameters
    ----------
    maxlen
        The maximum sequence length (inclusive).
    pad
        When set to True, input arrays will be zero-padded.
        The total size is determined by scipy.fft.next_fast_len.

    Returns
    -------
    timings
        An array with timings for input sizes 1 to maxlen.
    """
    rng = np.random.default_rng(1)
    data_maxlen = rng.uniform(0, 1, maxlen) + 1.0j * rng.uniform(0, 1, maxlen)
    timings = np.zeros(maxlen)
    for length in range(1, maxlen + 1):
        # Define a minimal namespace for the timeit function,
        namespace = {
            "data": data_maxlen[:length],
            "fft": fft,
            "length": next_fast_len(length) if pad else length,
        }
        timing = timeit.timeit("fft(data, length)", number=10, globals=namespace)
        timings[length - 1] = timing
    return timings


assert (time_fft(10) < 0.1).all()
assert (time_fft(10, True) < 0.1).all()

**Plotting function**

The following cell uses the functions `construct_primes` and `time_fft` to plot the cost of `fft` as a function of input size.

In [ ]:
def plot_timings_fft(maxlen):
    """Plot the timing of the FFT algorithm as a function of input size.

    Parameters
    ----------
    maxlen
        The maximum input size to consider.

    """
    sizes = np.arange(1, maxlen + 1)
    timings_normal = time_fft(maxlen, False)
    timings_padded = time_fft(maxlen, True)

    primes = construct_primes(maxlen)
    powers2 = 2 ** np.arange(int(np.floor(np.log2(maxlen))) + 1)

    plt.close("timing_fft")
    fig, axs = plt.subplots(1, 2, figsize=(7, 4), num="timing_fft", sharey=True)

    for ax, timings in zip(axs, [timings_normal, timings_padded], strict=False):
        ax.plot(sizes, timings, ".")
        ax.plot(primes, timings[primes - 1], ".", label="prime numbers")
        ax.plot(powers2, timings[powers2 - 1], "o", label="powers of 2")
        last_p2 = powers2[-1]
        scale = timings[last_p2 - 1] / (last_p2 * np.log2(last_p2))
        ax.plot(
            sizes[300:],
            sizes[300:] * np.log2(sizes[300:]) * scale,
            "--",
            label="Nlog(N)",
        )
        ax.set_xlabel("Input size [1]")
        ax.set_ylabel("Computation time [s]")
        ax.set_xscale("log")
        ax.set_yscale("log")
        ax.legend()
    axs[0].set_title("Timings without padding")
    axs[1].set_title("Timings with padding")


plot_timings_fft(4100)

## Applications

In many cases, the spectrum (in frequency domain) of a dataset is the quantity of interest, for which FFT is an obvious choice.
However, there are many more applications that, sometimes unexpectedly, rely on the Fast Fourier Transform.

### Signal processing

Signal processing is one of the domains where the DFT is used to manipulate streams of data.
For example, to filter out unwanted high-frequency noise from measurement data, one can take the FFT, set all high-frequency components to zero and then take the inverse transform.

Another example is the processing of data which contains multiple periodicities, which you want to investigate separately of each other.
E.g. a climate researcher might be interested in daily temperature fluctuations and wants to investigate them in the absence of the yearly cycle that lies on top of this data.
In such case, unwanted cycles can easily be removed in frequency domain, as shown in the example below.

> **Example**
> 
> The file `fft_weather.txt` contains hourly temperature data from Brussels airport from January 2010 until January 2021. (source: <https://mesonet.agron.iastate.edu/request/download.phtml?network=BE__ASOS>)
> We are interested in investigating yearly or daily temperature fluctuations, independently from each other.
> 
> First, let's see what the data looks like and whether we can identify these expected cycles.

In [ ]:
def plot_temperature():
    """Show the raw temperature data."""
    data = np.loadtxt("fft_weather.txt")
    plt.close("raw_temp")
    fig, ax = plt.subplots(num="raw_temp", figsize=(7, 4))
    ax.plot(data)
    ax.set_xlabel("Time [hour]")
    ax.set_ylabel("Temperature [°C]")


plot_temperature()

In [ ]:
def fft_temperature():
    """Visualize the spectrum of the temperature changes."""
    # Load the data and compute the FFT.
    data = np.loadtxt("fft_weather.txt")
    spectrum = rfft(data)
    freqs = rfftfreq(len(data), 1 / 24)

    # Plot absolute value of the DFT.
    # The amplitude is divided by len(spectrum) before plotting
    # to show the amplitude of the fluctuations on an
    # intuitive scale.
    plt.close("first_spectrum")
    fig, ax = plt.subplots(figsize=(7, 4), num="first_spectrum")
    ax.annotate(
        "daily\n cycle",
        xy=(1, 2),
        xytext=(1, 4),
        arrowprops=dict(facecolor="black", shrink=0.04),
    )
    ax.annotate(
        "yearly\n cycle",
        xy=(1 / 365.0, 7),
        xytext=(1 / 365, 9),
        arrowprops=dict(facecolor="black", shrink=0.04),
    )
    ax.plot(freqs, abs(spectrum) / len(spectrum), "-")
    ax.set_xlim(1e-4, 13)
    ax.set_ylim(-0.1, 10)
    ax.set_xscale("log")
    ax.set_xlabel("Frequency [1/day]")
    ax.set_ylabel("Amplitude [°C]")


fft_temperature()

> Now, let's try to get rid of the short-term fluctuations, such that the longer-term trends can be investigated more clearly.
> We'll set all frequency components above 100 days to zero.

In [ ]:
def filter_slow():
    """Retain only the slow fluctuations and plot the result."""
    # Load the data and compute the FFT.
    data = np.loadtxt("fft_weather.txt")
    spectrum = rfft(data)
    freqs = rfftfreq(len(data), 1 / 24)

    # Remove high-frequency data and transform back.
    # Suffix _f stands for "filtered".
    spectrum_f = spectrum * (freqs < 1 / 100)
    data_f = irfft(spectrum_f)

    plot_filter_results(freqs, data, spectrum, data_f, spectrum_f, "filter_slow")


def plot_filter_results(freqs, data, spectrum, data_f, spectrum_f, num):
    """Plot the original and filtered results.

    Parameters
    ----------
    freqs
        The frequency axis.
    data
        The hourly temperatures.
    spectrum
        The real FFT of the hourly temperatures.
    data_f
        The filtered hourly temperatures.
    spectrum_f
        The real FFT of the filtered hourly temperatures.
    num
        A figure number, which can also be a string.

    """
    plt.close(num)
    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(7, 4), num=num)
    ax0.annotate(
        "daily\n cycle",
        xy=(1, 2),
        xytext=(1, 4),
        arrowprops=dict(facecolor="black", shrink=0.04),
    )
    ax0.annotate(
        "yearly\n cycle",
        xy=(1 / 365.0, 7),
        xytext=(1 / 365, 9),
        arrowprops=dict(facecolor="black", shrink=0.04),
    )
    norm = len(spectrum)
    ax0.plot(freqs, abs(spectrum) / norm, alpha=0.2, label="original")
    ax0.plot(freqs, abs(spectrum_f) / norm, label="filtered")
    ax0.legend()
    ax0.set_xlim(1e-4, 13)
    ax0.set_ylim(-0.1, 10)
    ax0.set_xscale("log")
    ax0.set_xlabel("Frequency [1/day]")
    ax0.set_ylabel("Amplitude [°C]")

    ax1.plot(np.arange(len(data)) / 24, data, alpha=0.2)
    ax1.plot(np.arange(len(data_f)) / 24, data_f)
    ax1.set_xlabel("Time [day]")
    ax1.set_ylabel("Temperature [°C]")


filter_slow()

> Due to global warming, the input data is not exactly periodic and shows a slightly increasing trend.
> (This is on a time span of as little as 11 years!)
> This would be a typical example where the discrete cosine transform would be a slightly better algorithm.
>
> Let's now look at short-term fluctuations to get the typical Belgian daily temperature changes, without any longer-term fluctuations.
> For this, all fluctuations slower than 1 day will be set to zero.
> Only the first 10 days are plotted for the sake of clarity.

In [ ]:
def filter_fast():
    """Retain only the daily fluctuations and plot the result."""
    # Load the data and compute the FFT.
    data = np.loadtxt("fft_weather.txt")
    spectrum = rfft(data)
    freqs = rfftfreq(len(data), 1 / 24)

    # Remove low-frequency data and transform back.
    # Suffix _f stands for "filtered".
    spectrum_f = spectrum * (freqs >= 1.0)
    data_f = irfft(spectrum_f)

    plot_filter_results(
        freqs,
        data[: 24 * 10],
        spectrum,
        data_f[: 24 * 10],
        spectrum_f,
        "filter_fast",
    )


filter_fast()

### Efficient calculation of convolutions

The discrete circular convolution of two **periodic** sequences $\mathbf{u}$ and $\mathbf{v}$ of length $n$ is defined as

$$\{\mathbf{u*v}\}_m=\sum_{k=0}^{n-1}v_k u_{m-k},\quad \forall\, m \in \{0,1,\ldots,n-1\}.$$

The periodicity means that $v_k = v_{k + n}$ and $u_k = u_{k + n}$. Still, the arrays in computations only contain values for only one period.


This operation is equivalent to multiplication by a **circulant matrix**

$$
\begin{aligned} \begin{bmatrix}z_0\\z_1\\\vdots\\z_{n-2}\\z_{n-1}\end{bmatrix}=
\begin{bmatrix}
u_0&u_{n-1}&u_{n-2}&\cdots&u_1\\
u_1&u_{0}&u_{n-1}&\cdots&u_2\\
\vdots&\ddots&\ddots&\ddots&\vdots\\
u_{n-2}&\cdots&u_1&u_0&u_{n-1}\\
u_{n-1}&\cdots&u_2&u_1&u_{0}\\
\end{bmatrix}\begin{bmatrix}
v_0\\v_1\\\vdots\\v_{n-2}\\v_{n-1}
\end{bmatrix}
\end{aligned}
$$


Such a matrix is diagonalized by the DFT, thus: 

$$
\begin{aligned} \begin{bmatrix}\hat{z}_0\\\hat{z}_1\\\vdots\\\hat{z}_{n-2}\\\hat{z}_{n-1}\end{bmatrix}=
\begin{bmatrix}
\hat{u}_0&0&\cdots&\cdots&0\\
0&\hat{u}_{1}&0&\cdots&0\\
\vdots&\ddots&\ddots&\ddots&\vdots\\
0&\cdots&0&\hat{u}_{n-2}&0\\
0&\cdots&\cdots&0&\hat{u}_{n-1}\\
\end{bmatrix}\begin{bmatrix}
\hat{v}_0\\\hat{v}_1\\\vdots\\\hat{v}_{n-2}\\\hat{v}_{n-1}
\end{bmatrix}
\end{aligned}
$$

For this reason, it is more efficient to use the FFT algorithm to transform the inputs to the frequency domain, compute one pointwise multiplication, and transform the result back to the time domain.

> **Example**
>
>Let's calculate the convolution of the two sequences $\mathbf{u}=[0110]$ and $\mathbf{v}=[1100]$
>
> $$
\begin{aligned}\begin{bmatrix}
 z_0\\z_1\\z_2\\z_3\end{bmatrix}=\begin{bmatrix}
 0&0&1&1\\
 1&0&0&1\\
 1&1&0&0\\
 0&1&1&0\end{bmatrix}\begin{bmatrix}1\\1\\0\\0\end{bmatrix}=\begin{bmatrix}
 0\\1\\2\\1
 \end{bmatrix}\end{aligned}
$$
>
> Let's now verify this answer by using python to calculate
> 
> $$\mathbf{z}=\mathcal{F}^{-1}\left(\mathcal{F}(\mathbf{u})\mathcal{F}(\mathbf{v})\right)$$
>
> where $\mathcal{F}$ and $\mathcal{F}^{-1}$ denote the FFT and its inverse, respectively.
> 



In [ ]:
def demo_convolve_rfft():
    """Show how to compute a convolution with RFFT."""
    a = np.array([0, 1, 1, 0])
    b = np.array([1, 1, 0, 0])
    ffta = rfft(a)
    fftb = rfft(b)
    fftz = ffta * fftb
    # Note: the length is added as argument.
    # If not, it will always return a sequence of even length.
    z = irfft(fftz, len(a))
    # Note: suppress_small=True hides the rounding errors.
    print(np.array2string(z, suppress_small=True))


demo_convolve_rfft()

The same result could also be obtained with [`scipy.signal.convolve`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve.html).
We use the `method="direct"` option to make sure it does not use the FFTs for the calculation.
(By default, it will take the fastest method for the given lengths of the inputs.)

In [ ]:
def demo_convolve_scipy():
    """Compute the convolution with SciPy's convolve function."""
    a = np.array([0, 1, 1, 0])
    b = np.array([1, 1, 0, 0])
    z = signal.convolve(a, b, method="direct")
    print(z)


demo_convolve_scipy()

 The additional zero padding returned by `scipy.signal.convolve` becomes intuitively clear by viewing the convolution as the sum of the overlap, as a function of the shift, as the two sequences are shifted over each other.
 
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
  &   &   & 1 & 1 & 0 & 0 & \mathrm{pointwise} \\
0 & 1 & 1 & 0 &   &   &   & \mathrm{multiplication} \\ \hline
0 & 0 & 0 & 0 & 0 & 0 & 0 & \mathrm{sum}=0 \\ \hline
\end{array}
$$
> 
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
  &   & 1 & 1 & 0 & 0 & \\
0 & 1 & 1 & 0 &   &   & \\ \hline
0 & 0 & 1 & 0 & 0 & 0 & \mathrm{sum}=1 \\ \hline
\end{array}
$$
>
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
  & 1 & 1 & 0 & 0 & \\
0 & 1 & 1 & 0 &   & \\ \hline
0 & 1 & 1 & 0 & 0 & \mathrm{sum}=2 \\
\hline
\end{array}
$$
>
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
1 & 1 & 0 & 0 & \\
0 & 1 & 1 & 0 & \\ \hline
0 & 1 & 0 & 0 & \mathrm{sum}=1 \\
\hline
\end{array}
$$
>
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
1 & 1 & 0 & 0 &   & \\
  & 0 & 1 & 1 & 0 & \\ \hline
0 & 0 & 0 & 0 & 0 & \mathrm{sum}=0 \\ \hline
\end{array}
$$
>
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
1 & 1 & 0 & 0 &   &   & \\
  &   & 0 & 1 & 1 & 0 & \\ \hline
0 & 0 & 0 & 0 & 0 & 0 & \mathrm{sum}=0 \\ \hline
\end{array}
$$
>
> $$
\begin{array}{|c|c|c|c|c|c|c|c|c|c|c|c|}
\hline
1 & 1 & 0 & 0 &   &   &   & \\
  &   &   & 0 & 1 & 1 & 0 & \\ \hline
0 & 0 & 0 & 0 & 0 & 0 & 0 & \mathrm{sum}=0 \\ \hline
\end{array}
$$

The zero-padding can also be achieved with the FFT algorithm, by appropriately zero-padding the inputs.

In [ ]:
def demo_convolve_rfft_padding1():
    """Show how to compute a convolution with RFFT."""
    a = np.array([0, 1, 1, 0, 0, 0, 0])
    b = np.array([1, 1, 0, 0, 0, 0, 0])
    ffta = rfft(a)
    fftb = rfft(b)
    fftz = ffta * fftb
    z = irfft(fftz, len(a))
    print(np.array2string(z, suppress_small=True))


demo_convolve_rfft_padding1()

Zero-padding is also the correct method to compute convolutions of non-periodic inputs, assuming the data should be zero outside the range where it is defined.
Because this is such a common operation, the `fft` and `rfft` functions have an optional argument `n` to specify the desired zero-padding of the input.

In [ ]:
def demo_convolve_rfft_padding2():
    """Show how to compute a convolution with RFFT."""
    n = 7
    a = np.array([0, 1, 1, 0])
    b = np.array([1, 1, 0, 0])
    ffta = rfft(a, n)
    fftb = rfft(b, n)
    fftz = ffta * fftb
    z = irfft(fftz, n)
    print(np.array2string(z, suppress_small=True))


demo_convolve_rfft_padding2()

### Autocorrelation

The **autocorrelation** of a real sequence $\mathbf{y}$ expresses the similarity between a sequence and a delayed copy of itself.
It is defined as

$$
\mathbf{R}_\ell=\sum_{k=0}^{n-1} \mathbf{y}_k\mathbf{y}_{k-\ell}\quad \forall\, l\in\{0,1,\ldots,n-1\},
$$
 
We recognize that this is a convolution of the sequence with a **reversed copy** of itself, i.e. $\mathbf{y}_{k-\ell}$ instead of $\mathbf{y}_{\ell-k}$, and thus can be calculated efficiently using FFTs as

$$
\mathbf{R}
=\mathcal{F}^{-1}\left(\mathcal{F}(\mathbf{y})(\mathcal{F}(\mathbf{y})^*)\right)
=\mathcal{F}^{-1}\left(\Vert\mathcal{F}(\mathbf{y})\Vert^2\right)
$$

where $^*$ denotes a complex conjugate.
The complex conjugation is due to the reversal of the second convolution argument in the time domain.

If we want to get the **autocovariance** instead of the **autocorrelation**, we need to *demean* (i.e. subtracting the mean of the sequence) the sequence:

$$\mathbf{K}=\mathcal{F}^{-1}\left(\Vert\mathcal{F}(\mathbf{y} - \overline{\mathbf{y}})\Vert^2\right)$$

It is also common to further divide by the *variance* of the sequence, which is conveniently calculated from the first entry in the Fourier transformed sequence.

$$\mathbf{\rho}=\frac{\mathbf{K}}{\sigma_y^2}$$

(There is no proper name for the latter quantity. It is often called autocorrelation or autocovariance, while it is actually neither.)

> **Example**
>
> As an example, we'll generate a random sequence where each number is determined by taking 95% of the previous number and adding 5% of a random number to this value.
> Note that we generate our random values between -1 and 1, so that the mean will be 0.
>
> We thus expect that the correlation between each value and a value $n$ positions further scales as $0.95^n$. 

In [ ]:
def demo_autocovariance_fft():
    """Demonstrate the computation of an autocovariance with RFFT."""
    # Generate the stochastic input.
    # The calculation is unavoidably sequential,
    # so no vectorization possible.
    rng = np.random.default_rng()
    a = np.zeros(100000)
    a[0] = rng.uniform(-1, 1)
    for i in range(1, len(a)):
        a[i] = a[i - 1] * 0.95 + 0.05 * rng.uniform(-1, 1)

    # Compute auto-correlation.
    # In principle, we would have to use padding,
    # but it has no noticeable effect in practice.
    y = rfft(a)
    R = irfft(abs(y) ** 2)

    # Compare to the analytical result in a plot.
    x = np.arange(150)
    f = (0.95) ** x

    plt.close("autocovariance")
    fig, ax = plt.subplots(figsize=(7, 4), num="autocovariance")
    ax.plot(x, R[: len(x)] / R[0], "-", label="autocovariance")
    ax.plot(x, f, "-", label="$0.95^x$")
    ax.set_xlabel("n")
    ax.set_ylabel("Autocovariance")
    plt.legend()


demo_autocovariance_fft()

There are many more physical quantities which used to be very computationally demanding to compute, but which can luckily be computed (a bit) more efficiently using the Fast Fourier Transform algorithm (although typically, these computations remain the bottleneck in simulation programs). 

One example is the long-range magnetostatic interaction, which in a naive implementation requires $\mathcal{O}(n^2)$ calculations. ($n$ interactions for each of the $n$ spins in the system).
By recasting the equations in a suitable way, this can be written as a convolution, which allows for a more efficient computation.

### Fast polynomial multiplication

Another example of a calculation that can be sped up using FFT's is the multiplication of two polynomial functions.

To find the coefficients of the product of two polynomials $f(x)=f_1(x) \cdot f_2(x)$, we need to:
- calculate all the pair-wise products of their respective terms,
- group these products by the order of the resulting mononomial,
- and sum the products within each group.

When we write the coefficients as vectors $\mathbf{f}_1$ and $\mathbf{f}_2$ (ordered from the lowest order to the highest; i.e. the constant term first), and append zeros such that the vectors have a dimension larger than the degree of $f_1$ + the degree of $f_2$, we can write the polynomial product as a convolution, where the coefficients of their product $\mathbf{f}$ are given by:

$$\mathbf{f}=\mathbf{f}_1*\mathbf{f}_2$$

We saw earlier that we can calculate this in an efficient way using Fast Fourier Transforms: 

$$\mathbf{f}=\mathcal{F}^{-1}\bigl(\mathcal{F}(\mathbf{f}_1)\mathcal{F}(\mathbf{f}_2)\bigr)$$

> **Example**
>
> Let's try to find the product of the functions
> 
> $$f_1=3+2x+x^2+x^3$$
>
> and
>
> $$f_2=1+x+x^2+x^3.$$
>
> The corresponding coefficient vectors are
>
> $$
\mathbf{f}_1= [3,2,1,1,0,0,0]
\quad\text{and}\quad
\mathbf{f}_2= [1,1,1,1,0,0,0].
$$
>
> Note the zero padding to the size needed to represent the product (up to $x^6$).
>
> 
> As shown below, we find that the product of $f_1$ and $f_2$ equals
> 
> $$
\begin{aligned}
    \mathbf{f}
    &= \mathcal{F}^{-1}\bigl(\mathcal{F}(\mathbf{f}_1)\mathcal{F}(\mathbf{f}_2)\bigr) \\
    &= [3,5,6,7,4,2,1]
\end{aligned}
$$
>
> where $\mathcal{F}$ and $\mathcal{F}^{-1}$ denote the FFT and its inverse, respectively.
> Thus:
>
> $$f(x)=f_1(x)\cdot f_2(x)=3+5x+6x^2+7x^3+4x^4+2x^5+1x^6$$

In [ ]:
def multiply_polys(a1, a2):
    """Multiply two polynomials, whose coefficients are given in a1 and a1."""
    # Zero padding is used to make sure the output is
    # sufficiently large for the term with the highest degree.
    length = len(a1) + len(a2) - 1
    y1 = rfft(a1, length)
    y2 = rfft(a2, length)
    return irfft(y1 * y2, length)


def eval_poly(a, x):
    """Evaluate a polynomial in with coefficients a in point x."""
    # The following is the product of a column
    # from a Vandermonde matrix and a vector.
    return np.dot(x ** np.arange(len(a)), a)


def demo_poly():
    """Demonstrate polynomial multiplication through RFFT."""
    # you can check for yourself that for any x, you'll get the correct result
    x = 3.0
    a1 = np.array([3, 2, 1, 1])
    a2 = np.array([1, 1, 1, 1])
    a3 = multiply_polys(a1, a2)
    print("Coefficients of polynomial product:", a3)
    print("Evaluate product of polynomials:", eval_poly(a1, x) * eval_poly(a2, x))
    print("Evaluate polynomial product:", np.abs(eval_poly(a3, x)))


demo_poly()

## SciPy resources

All details of the `fft`, `rfft` and related functions we used throughout this notebook can be found on

<https://docs.scipy.org/doc/scipy/reference/fft.html>

Furthermore, an instructive tutorial in the `scipy` user guide is available on

<https://docs.scipy.org/doc/scipy/tutorial/fft.html>